## TransferLearning

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os

import numpy as np

import matplotlib.pyplot as plt

from keras.preprocessing.image import ImageDataGenerator

from keras.models import Sequential

from keras.layers import Conv2D, MaxPooling2D

from keras.layers import Activation, Dropout, Flatten, Dense

from keras import backend as K

from keras.layers import BatchNormalization

from keras.layers import LeakyReLU

Using TensorFlow backend.


In [3]:
import tensorflow as tf

from tensorflow import keras

In [4]:
IMG_SIZE = 150
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)


base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


C:\Users\DELL\Anaconda3\lib\site-packages\keras_applications\mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


In [38]:

train_data_dir = r'E:/Final project/train/Phytoplankton'
img_height, img_width = 150, 150
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 1
batch_size = 32

train_datagen = ImageDataGenerator(rescale=1./255,
     rotation_range=40,
     width_shift_range=0.2,
     height_shift_range=0.2,
     shear_range=0.2,
     zoom_range=0.2,
     horizontal_flip=True,
     validation_split=0.2) # set validation split

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    shuffle=True,
    classes = ['artifacts', 'Chaetognaths', 'Copepods', 'Ctenophores', 'detritus', 'diatom', 'Echinoderms', 'Fish','Gastropods', 'hydromedusae', 'Other Invert Larvae', 'Pelagic Tunicates', 'Protists', 'Shrimp', 'Siphonophores', 'trichodesmium'],
    subset='training')


validation_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    shuffle=True,
    classes = ['artifacts', 'Chaetognaths', 'Copepods', 'Ctenophores', 'detritus', 'diatom', 'Echinoderms', 'Fish','Gastropods', 'hydromedusae', 'Other Invert Larvae', 'Pelagic Tunicates', 'Protists', 'Shrimp', 'Siphonophores', 'trichodesmium'],
    subset='validation')



if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)


train_batches = next(train_generator)
validation_batches = next(validation_generator)

feature_batch = base_model(train_batches)
print(feature_batch.shape)

Found 22017 images belonging to 16 classes.
Found 5495 images belonging to 16 classes.
(32, 5, 5, 1280)


In [9]:
base_model.trainable = False
base_model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 151, 151, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 75, 75, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 75, 75, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________

Total params: 2,257,984
Trainable params: 0
Non-trainable params: 2,257,984
__________________________________________________________________________________________________


In [10]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

(32, 1280)


In [11]:
prediction_layer = keras.layers.Dense(16)
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

(32, 16)


In [31]:
model8 = tf.keras.Sequential([
  base_model,
  global_average_layer, 
])
model8.add(keras.layers.Dense(128))
model8.add(keras.layers.Dense(64))
model8.add(keras.layers.LeakyReLU(alpha=0.1))
model8.add(keras.layers.Dropout(0.3))
model8.add(keras.layers.Dense(16))
model8.add(keras.layers.Activation('softmax'))

In [32]:
base_learning_rate = 0.001
model8.compile(optimizer=tf.keras.optimizers.RMSprop(lr=base_learning_rate),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'], verbose = 1)

In [33]:
model8.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Model) (None, 5, 5, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense_11 (Dense)             (None, 128)               163968    
_________________________________________________________________
dense_12 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 16)                1040      
_________________________________________________________________
activation_2 (Activation)    (None, 16)               

In [34]:
len(model8.trainable_variables)

6

In [35]:
initial_epochs = 10
validation_steps=20
batch_size = 32
loss0,accuracy0 = model.evaluate(validation_generator, steps = validation_steps)

20/20 [==============================] - 20s 976ms/step - loss: 2.7553 - acc: 0.1125


In [36]:
print("initial loss: {:.2f}".format(loss0))
print("initial accuracy: {:.2f}".format(accuracy0))

initial loss: 2.76
initial accuracy: 0.11


In [37]:
model8.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

Epoch 1/10
62/62 [==============================] - 218s 4s/step - loss: 2.5685 - acc: 0.3044 - val_loss: 2.6491 - val_acc: 0.2188
Epoch 2/10
62/62 [==============================] - 208s 3s/step - loss: 2.4528 - acc: 0.4204 - val_loss: 2.4772 - val_acc: 0.3950
Epoch 3/10
62/62 [==============================] - 213s 3s/step - loss: 2.3495 - acc: 0.5232 - val_loss: 2.4714 - val_acc: 0.4062
Epoch 4/10
62/62 [==============================] - 211s 3s/step - loss: 2.3377 - acc: 0.5418 - val_loss: 2.4910 - val_acc: 0.3738
Epoch 5/10
62/62 [==============================] - 209s 3s/step - loss: 2.3179 - acc: 0.5570 - val_loss: 2.5819 - val_acc: 0.2887
Epoch 6/10
62/62 [==============================] - 213s 3s/step - loss: 2.3125 - acc: 0.5640 - val_loss: 2.4873 - val_acc: 0.3887
Epoch 7/10
62/62 [==============================] - 210s 3s/step - loss: 2.3207 - acc: 0.5534 - val_loss: 2.5174 - val_acc: 0.3512
Epoch 8/10
62/62 [==============================] - 211s 3s/step - loss: 2.2839 - a

In [ ]:
model8.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)